In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import tarfile
import io
import re
import subprocess as sp

from typing import Dict, List, Tuple, Union, Optional

from loguru import logger


In [ ]:
import pandas as pd
import numpy as np

import waac
import waac.config as config
from sklearn.decomposition import NMF

In [ ]:
RANDOM_SEED = 6545

In [ ]:
default_rng = np.random.default_rng(seed=RANDOM_SEED)

In [ ]:
DATA_DIR = config.DATA_DIR

In [ ]:
fp = DATA_DIR / "intermediate" / "netflix_to_imdb.csv"

id_mapping = pd.read_csv(fp).dropna(subset="tconst")

In [ ]:
id_mapping

In [ ]:
n = 10
downsample_id_mapping = id_mapping.sample(n, random_state=RANDOM_SEED)

In [ ]:
s = downsample_id_mapping.movie_ID.iloc[0]
str(s).zfill(7)

In [ ]:
for movie_id in downsample_id_mapping.movie_ID:
    training_fp = DATA_DIR / "download" / "training_set" / f"mv_{str(movie_id).zfill(7)}.txt"
    print(training_fp)
    print(training_fp.exists())
    

In [ ]:
training_tar_fp = DATA_DIR / "download" / "training_set.tar"

with tarfile.open(training_tar_fp) as tf:
    names = tf.getnames()
    # "training_set/mv_1234567.txt" ...
print(names)

In [ ]:
training_tar_fp = DATA_DIR / "download" / "training_set.tar"

with tarfile.open(training_tar_fp) as tf:
    for movie_id in downsample_id_mapping.movie_ID:
        member = f"training_set/mv_{str(movie_id).zfill(7)}.txt"
        print(f"{member=}")
        x = tf.extractfile(member)
        print(x)
        print(type(x))
        # read header row
        y0 = next(x)
        # read the rest
        d = pd.read_csv(x, encoding="utf-8", header=None, names=config.raw_data_column_names["training_set"]["movie_ID"])
        # y = [_ for _ in x]
        # contents = x.read().decode("utf-8")
        break

    

In [ ]:
d

# todo:
# add a column called movieID, spread the df so customer is index, movieID is columns, values = rating. (date is removed)
# maybe have to drop duplicates on [customer_ID,movieID] first incase someone rated the same movie twice

# This is the input for NMF

In [ ]:
a = np.array([4,5,5,np.NaN, 5,5, np.NaN, np.NaN, np.NaN]].reshape(3,3)
a

In [ ]:
y[:4]

In [ ]:
y0

In [ ]:
contents

In [ ]:
_bytes.decode("utf-8")

In [ ]:
list((DATA_DIR / "download" / "training_set").glob("*.txt"))